In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas as pd
import numpy as np

In [3]:
import json

In [59]:
with open('dac_match_details.txt', 'r') as fi:
    buf = fi.read()
data = json.loads(buf)

In [60]:
df = pd.DataFrame(data)

In [14]:
df[df['radiant_name'].map(lambda x:x=='Vici Gaming')]

,barracks_status_dire,barracks_status_radiant,cluster,dire_captain,dire_guild_id,dire_guild_logo,dire_guild_name,dire_logo,dire_name,dire_team_complete,...,radiant_guild_logo,radiant_guild_name,radiant_logo,radiant_name,radiant_team_complete,radiant_team_id,radiant_win,start_time,tower_status_dire,tower_status_radiant
0,63,3,223,86727555,NaN,NaN,NaN,5.430139e+17,Evil Geniuses,1,...,NaN,NaN,3.409695e+16,Vici Gaming,1,726228,False,1423464603,1974,1542
2,59,15,223,86727555,NaN,NaN,NaN,5.430139e+17,Evil Geniuses,1,...,NaN,NaN,3.409695e+16,Vici Gaming,1,726228,False,1423472994,1924,1572
10,60,63,223,86727555,NaN,NaN,NaN,5.430139e+17,Evil Geniuses,1,...,NaN,NaN,3.409695e+16,Vici Gaming,1,726228,True,1423367760,1824,1846
33,63,60,223,87278757,NaN,NaN,NaN,5.407680e+17,Team Secret,1,...,NaN,NaN,3.409695e+16,Vici Gaming,1,726228,False,1423197718,1972,1952
36,3,63,223,87432909,NaN,NaN,NaN,2.622454e+16,HellRaisers`,0,...,NaN,NaN,3.409695e+16,Vici Gaming,1,726228,True,1423147242,1542,1972
76,47,63,223,140153524,NaN,NaN,NaN,4.698101e+17,CDEC Gaming,0,...,NaN,NaN,3.409695e+16,Vici Gaming,1,726228,True,1422801420,1572,2047
90,0,63,223,43276219,NaN,NaN,NaN,6.139166e+17,Cloud9,1,...,NaN,NaN,3.409695e+16,Vici Gaming,1,726228,True,1422774787,1536,1976
106,51,63,223,89137399,NaN,NaN,NaN,9.201104e+17,Natus Vincere,1,...,NaN,NaN,3.409695e+16,Vici Gaming,1,726228,True,1422705848,1796,1846
111,51,63,223,87432909,NaN,NaN,NaN,2.622454e+16,HellRaisers`,0,...,NaN,NaN,3.409695e+16,Vici Gaming,1,726228,True,1422694194,1796,1975
128,48,63,223,91162296,NaN,NaN,NaN,5.779028e+17,R a v e,1,...,NaN,NaN,3.409695e+16,Vici Gaming,1,726228,True,1422630490,1792,1828


In [26]:
print(type(df['radiant_name']))
print(type(df[['radiant_name', 'dire_name']]))
#使用Boolean index结合any方法，选择参赛队伍名称为Vici Gaming的比赛
team_frame = df[(df[['radiant_name', 'dire_name']]=='Vici Gaming').any(1)]

<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>


In [29]:
print(type(team_frame))
team_frame['match_id']

<class 'pandas.core.frame.DataFrame'>


0      1223581106
1      1223686866
2      1223796316
9      1220593906
10     1220694122
32     1215341090
33     1215457506
34     1215530933
35     1214159762
36     1214292333
60     1206554979
62     1206412121
69     1206150223
76     1204856693
84     1204148113
90     1203745463
106    1201583051
111    1201132306
128    1199398898
137    1198703503
142    1198410243
150    1198116859
159    1196830939
167    1196271183
172    1196011576
Name: match_id, dtype: int64

In [48]:
account_id = 101695162
def getKAD(row):
    for x in row['players']:
        if x['account_id']==account_id:
            if x['deaths']==0:
                return (x['kills']+x['assists'])/1.0
            else:
                return (x['kills']+x['assists'])*1.0/x['deaths']
#使用DataFrame的
kads = [getKAD(row) for index, row in team_frame.iterrows()]
print(kads)

[1.8333333333333333, 2.0, 1.1111111111111112, 3.5, 3.8, 13.0, 1.8333333333333333, 23.0, 3.6666666666666665, 5.666666666666667, 1.8, 2.7142857142857144, 3.8333333333333335, 23.0, 11.0, 6.666666666666667, 3.0, 6.5, 2.5, 17.0, 1.2857142857142858, 5.0, 18.0, 12.0, 1.4]


In [61]:
#参赛人数为10的队长模式的比赛
cap_frame = df[(df['players'].map(lambda x:len(x)==10))&(df['game_mode']==2)]
len(cap_frame)

476

In [94]:
#使用reduce方法计算队伍的总击杀数
players = cap_frame.iloc[0]['players'][:5]
def add(x, y):
    if isinstance(x, dict):
        return x['kills']+y['kills']
    return x+y['kills']
reduce(add, players)

12

In [100]:
#计算kad和参战率
def getTeamKills(x, y):
    return x+y['kills']
for index, row in cap_frame.iterrows():
    players = row['players']
    radiant_kills = reduce(getTeamKills, players[:5], 0)
    dire_kills = reduce(getTeamKills, players[5:], 0)
    for player in players:
        ka = (player['kills']+player['assists'])*1.0
        #计算kad
        if player['deaths']==0:
            player['kad'] = ka
        else:
            player['kad'] = ka/player['deaths']
        #计算参战率    
        if player['player_slot']<5:
            if radiant_kills==0:
                player['rate'] = 0
            else:
                player['rate'] = ka/radiant_kills
        else:
            if dire_kills==0:
                player['rate'] = 0
            else:
                player['rate'] = ka/dire_kills

In [101]:
cap_frame['players'][0]

[{u'ability_upgrades': [{u'ability': 5448, u'level': 1, u'time': 909},
   {u'ability': 5450, u'level': 2, u'time': 1035},
   {u'ability': 5450, u'level': 3, u'time': 1223},
   {u'ability': 5448, u'level': 4, u'time': 1394},
   {u'ability': 5450, u'level': 5, u'time': 1590},
   {u'ability': 5452, u'level': 6, u'time': 1809},
   {u'ability': 5450, u'level': 7, u'time': 1898},
   {u'ability': 5448, u'level': 8, u'time': 1939},
   {u'ability': 5448, u'level': 9, u'time': 1998},
   {u'ability': 5451, u'level': 10, u'time': 2191},
   {u'ability': 5452, u'level': 11, u'time': 2559},
   {u'ability': 5451, u'level': 12, u'time': 3103},
   {u'ability': 5451, u'level': 13, u'time': 3218}],
  u'account_id': 101695162,
  u'assists': 7,
  u'deaths': 6,
  u'denies': 4,
  u'gold': 51,
  u'gold_per_min': 216,
  u'gold_spent': 8545,
  u'hero_damage': 5083,
  u'hero_healing': 0,
  u'hero_id': 86,
  u'item_0': 1,
  u'item_1': 36,
  u'item_2': 46,
  u'item_3': 23,
  u'item_4': 21,
  u'item_5': 180,
  'kad'